# SQL pré-armazenado: stored procedures, triggers, views

Além de armazenar nossos dados em tabelas, os bancos de dados também podem armazenar código SQL na forma de vários tipos de objetos:

- **Stored procedures**: procedimentos escritos em SQL, executados através da chamada `CALL`.
- **Stored functions**: funções escritas em SQL e que podem ser usadas nas mesmas situações que uma função pré-definida seria usada, como `SUM()` ou `COUNT()`
- **Triggers**: Um procedimento que será executado automaticamente quando determinadas condições ocorrem, como `INSERT`, `UPDATE` ou `DELETE` em uma tabela.
- **Eventos**: Procedimentos que podem ser executados pelo banco de dados em horários pré-definidos.
- **Views**: Diferente dos outros objetos, uma view é como um `SELECT` pré-definido, e resulta em uma tabela virtual.

Vamos continuar trabalhando com a base de dados 'emprestimo' de uma aula anterior. Para facilitar a atividade de hoje, vamos começar do zero rodando o script 'emprestimos.sql':

```SQL
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);
```

Em seguida, vamos abrir a conexão com o banco de dados.

In [13]:
from functools import partial

import mysql.connector


def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            print("Executando query:")
            cursor.execute(query, args)
            for result in cursor:
                print(result)

    connection = mysql.connector.connect(
        host="localhost",
        user="megadados",
        password="megadados",
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper("emprestimos")

## Stored procedures

Existem algumas vantagens em procedimentos armazenados (stored procedures):

- *Reuso de código*: como em qualquer outra linguagem de programação, podemos construir stored procedures para definir tarefas rotineiras e reutilizá-las em vários workflows.
- *Segurança*: Ao invés de oferecer acesso direto à tabelas, podemos bloquear todos os acessos a tabelas e definir acesso à stored procedures! Assim, ao invés de permitir acesso à tabela *usuario*, podemos permitir acesso ao procedimento *saque()* ou *consulta_saldo()*
- *Desempenho*: ao invés de mandar vários comandos SQL para a base de dados toda vez que quisermos realizar determinada tarefa, podemos simplesmente chamar um procedimento armazenado, pois todo o código já está na base de dados. Ademais, a base de dados não precisa compilar o procedimento toda vez que este é chamado, basta compilar na primeira execução e manter o código compilado em um cache.

Existem também desvantagens:

- *Debugging*: pode ser difícil debugar um procedimento armazenado
- *Portabilidade*: a sintaxe de definição de stored procedures é raramente portável entre diferentes sistemas de gerenciamento de bancos de dados.
- *Separação entre dados e lógica de negócios*: com stored procedures estamos migrando parte da lógica de negócios para o banco de dados. Alterações na lógica agora demandam modificações (e manutenção) em partes distintas da sua aplicação, uma receita para dor de cabeça!
- *Aumento da carga de processamento no servidor*: servidores de banco de dados costumam ser otimizados para memória e largura de banda, não para processamento

Ou seja, use procedimentos armazenados com cautela!

### Interlúdio: o valor da experiência

Como você já deve ter percebido à essa altura do curso, você tem um camilho longo pela frente, desde conhecer um conceito novo até dominar o uso deste conceito com sabedoria - isso chama-se ganhar experiência! Não subestime o valor de trabalhar "nas trincheiras" aplicando seus conhecimentos novos e aprendendo com a experiência dos veteranos. Além disso, continue se aperfeiçoando - faça cursos para conhecer melhor a tecnologia, invente projetos para tentar novas ideias, assista vídeos no YouTube sobre melhores práticas, busque projetos (e empregos) onde você possa se desenvolver.

É impossível formar engenheiros experientes em um curso - mesmo que fossem dois cursos, não cobriríamos tudo que existe sobre banco de dados. Mas estou certo de que vocês adquiriram novas ferramentas neste módulo do curso, e estão capacitados a continuar progredindo em banco de dados.

Nós, os professores do Insper, estamos fazendo o possível para transferir a vocês mais do que conhecimento - estamos buscando desenvolver competência também! E para isso só existe um caminho: a prática. Façam exercícios, coloquem esforço nos projetos, e venham preparados para a aula para que possamos ir além da mera leitura do material didático, ok?

Voltamos agora à programação regular...

### Construindo uma stored procedure

Vamos começar com uma stored procedure para adicionar um usuário. Crie e execute o seguinte script no MySQL workbench:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS adiciona_usuario;

DELIMITER //
CREATE PROCEDURE adiciona_usuario(IN novo_nome VARCHAR(80), IN novo_sobrenome VARCHAR(80))
BEGIN
    INSERT INTO usuario (nome, sobrenome) VALUES (novo_nome, novo_sobrenome);
END//
DELIMITER ;
```

Note o uso destes comandos `DELIMITER`. Coisas de MySQL: ele não entende que os ponto-e-virgula internos ao procedimento não sinalizam o final do comando CREATE PROCEDURE...

Agora podemos usar nossa nova procedure para adicionar alguns usuários!

In [2]:
db('SELECT * FROM usuario')

Executando query:
(1, 'Juca', 'Silva', Decimal('12000.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('1000.00'))
(4, 'Antonio', 'Reis', Decimal('11000.00'))
(5, 'Paulo', 'Oliveira', Decimal('11000.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))


In [3]:
#db("START TRANSACTION;")
#try:
#    db("CALL adiciona_usuario('Juca', 'Silva');")
#    db("CALL adiciona_usuario('Mario', 'Ferreira');")
#    db("CALL adiciona_usuario('Ana', 'Soares');")
#    db("CALL adiciona_usuario('Antonio', 'Reis');")
#    db("CALL adiciona_usuario('Paulo', 'Oliveira');")
#    db("COMMIT;")
#except Exception as e:
#    print(e)
#    db("ROLLBACK");

In [3]:
db('START TRANSACTION')
try:
    db("CALL adiciona_usuario('Max', 'Verstappen');")
    db("CALL adiciona_usuario('Lando', 'Norris');")
    db("CALL adiciona_usuario('Charles', 'Leclerc');")
except Exception as e:
    print(e)
    db('ROLLBACK');

Executando query:
Executando query:
Executando query:
Executando query:


Vamos verificar se tudo funcionou bem:

In [4]:
db("SELECT * FROM usuario")

Executando query:
(1, 'Juca', 'Silva', Decimal('12000.00'))
(2, 'Mario', 'Ferreira', Decimal('0.00'))
(3, 'Ana', 'Soares', Decimal('1000.00'))
(4, 'Antonio', 'Reis', Decimal('11000.00'))
(5, 'Paulo', 'Oliveira', Decimal('11000.00'))
(6, 'Carla', 'Nakamura', Decimal('0.00'))
(7, 'Mario', 'Fontana', Decimal('0.00'))
(10, 'Max', 'Verstappen', Decimal('0.00'))
(11, 'Lando', 'Norris', Decimal('0.00'))
(12, 'Charles', 'Leclerc', Decimal('0.00'))


### Stored functions

Agora vamos criar uma função para consultar o saldo. Rode o script a seguir no MySQL Workbench:

```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS saldo;

DELIMITER //
CREATE FUNCTION saldo(id INT) RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
	DECLARE saldo_procurado DECIMAL(30, 2);
	SELECT IFNULL(saldo, 0.0) INTO saldo_procurado FROM usuario WHERE id_usuario = id;
    RETURN saldo_procurado;
END//
DELIMITER ;
```

Vamos testar esta função:

In [5]:
db("SELECT saldo(1)")

Executando query:
(Decimal('12000.00'),)


In [6]:
db('INSERT INTO usuario (nome, sobrenome, saldo) VALUES ("Walter", "Branco", 20.50)')
db('COMMIT')

Executando query:
Executando query:


In [8]:
db("SELECT saldo(13)")

Executando query:
(Decimal('20.50'),)


### Atividades

Faça um script SQL que cria uma stored procedure para cobrar uma taxa de manutenção de D dinheiros de cada conta, desde que a conta tenha saldo. (Dica: use a função `IF()`: https://dev.mysql.com/doc/refman/8.0/en/flow-control-functions.html#function_if)

``` mysql

USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

DELIMITER //
CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30,2), IN id_usuario_procurado INT)
BEGIN
    DECLARE taxa_cobrada DECIMAL(30,2);
    SELECT IF(saldo(id_usuario_procurado)>0.0, taxa, 0.0) INTO taxa_cobrada;
    UPDATE usuario SET saldo = saldo - taxa WHERE id_usuario = id_usuario_procurado;
END//
DELIMITER ;

```


**Resposta**:

```SQL
USE emprestimos;

DROP PROCEDURE IF EXISTS cobra_taxa;

DELIMITER //
CREATE PROCEDURE cobra_taxa(IN taxa DECIMAL(30, 2))
BEGIN
    --- Gambiarra para desligar o aviso de segurança que previne
    --- UPDATES gerais (sem WHERE).
    SET @OLD_SQL_SAFE_UPDATES = @@SQL_SAFE_UPDATES;
    SET SQL_SAFE_UPDATES=0;

    --- Insira seu código aqui.
    
    --- Desliga a gambiarra.
    SET SQL_SAFE_UPDATES=@OLD_SQL_SAFE_UPDATES;
END //
DELIMITER ;
```

Escreva um código SQL para testar sua procedure:

In [9]:
query = "CALL cobra_taxa(200.00, 3);"
with connection.cursor() as cursor:
    cursor.execute(query)
    for result in cursor:
        print(result)

In [10]:
db('ROLLBACK')

Executando query:


**Resposta:**

Faça uma função que retorna a soma de todos os saldos da base de dados.

``` mysql

USE emprestimos;

DROP FUNCTION IF EXISTS total_saldo;

DELIMITER //
CREATE FUNCTION total_saldo() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN
    DECLARE soma_total DECIMAL(30, 2);
    SELECT 
        SUM(saldo) INTO soma_total 
    FROM usuario;
    RETURN soma_total;
END//
DELIMITER ;

```

**Resposta**:

```SQL
USE emprestimos;

DROP FUNCTION IF EXISTS total_saldos;

DELIMITER //
CREATE FUNCTION total_saldos() RETURNS DECIMAL(30, 2) READS SQL DATA
BEGIN

    -- Insira seu SQL aqui.

END//
DELIMITER ;
```

Escreva um código SQL para testar sua função:

In [14]:
db('SELECT total_saldo()')

Executando query:
(Decimal('34720.50'),)


**Resposta:**

## Triggers

*Triggers*, ou gatilhos, são procedimentos armazenados que são executados automaticamente quando uma operação é realizada. Você pode escolher se o trigger ocorre antes ou depois da ação.

Vamos fazer um gatilho para atualizar automaticamente o saldo do usuário quando uma nova movimentação é inserida na tabela `movimentacao`. Vamos também criar uma *constraint* para impedir que um saldo negativo exista na nossa base de dados.

Bom, rode o script SQL a seguir no MySQL Workbench:

```SQL
USE emprestimos;

DROP TRIGGER IF EXISTS trig_movimentacao;

DELIMITER //
CREATE TRIGGER trig_movimentacao 
BEFORE INSERT ON movimentacao
FOR EACH ROW
BEGIN
    UPDATE usuario 
        SET saldo = saldo + NEW.valor 
        WHERE id_usuario = NEW.id_usuario;
END//

DELIMITER ;
```

*Interlúdio*: e quanto ao constraint `CHECK`? No MySQL 5.7 o `CHECK` não funcionava, apesar de previsto no padrão SQL. Na versão 8.0 isso foi corrigido.

Na documentação do MySQL 5.7 (https://dev.mysql.com/doc/refman/5.7/en/create-table.html) temos o seguinte:

    The `CHECK` clause is parsed but ignored by all storage engines. See Section 1.8.2.3, “Foreign Key Differences”.

A título de curiosidade: o remédio era criar um trigger para evitar a atualização com valores errados. Veja a gambiarra em ação no código a seguir.

```SQL
USE emprestimos;

DROP TRIGGER IF EXISTS trig_saldo_insuficiente;

CREATE TRIGGER trig_saldo_insuficiente 
BEFORE UPDATE ON usuario
FOR EACH ROW
BEGIN
    -- https://stackoverflow.com/questions/2115497/check-constraint-in-mysql-is-not-working
    IF NEW.saldo < 0.0 THEN
        SIGNAL SQLSTATE '12345'
            SET MESSAGE_TEXT = 'Saldo insuficiente.';
    END IF;
END//
DELIMITER ;
```

Já na versão atual do MySQL o check de saldo pode ser implementado normalmente com o constraint `CHECK`

```SQL
USE emprestimos;
ALTER TABLE usuario ADD CONSTRAINT chk_saldo CHECK (saldo >= 0.0);
ALTER TABLE emprestimo ADD CONSTRAINT chk_emprestimo CHECK (valor_atual >= 0.0);
```

Vamos testar esse trigger na nossa tabela:

In [15]:
def movimentacao(id_usuario, valor):
    db(
        "INSERT INTO movimentacao (id_usuario, valor) VALUES (%s, %s)",
        (id_usuario, valor),
    )
    db("SELECT * FROM usuario WHERE id_usuario=%s", (id_usuario,))

In [16]:
db("START TRANSACTION")
movimentacao(1, 1000)

Executando query:
Executando query:
Executando query:
(1, 'Juca', 'Silva', Decimal('13000.00'))


In [17]:
db('select * from movimentacao')

Executando query:
(1, 3, Decimal('10000.00'), datetime.datetime(2022, 12, 3, 15, 35, 27))
(2, 4, Decimal('5000.00'), datetime.datetime(2022, 12, 3, 15, 35, 27))
(3, 5, Decimal('20000.00'), datetime.datetime(2022, 12, 3, 15, 35, 27))
(34, 3, Decimal('-1000.00'), datetime.datetime(2022, 12, 3, 15, 58, 49))
(35, 1, Decimal('1000.00'), datetime.datetime(2022, 12, 3, 15, 58, 49))
(36, 3, Decimal('-2000.00'), datetime.datetime(2022, 12, 3, 15, 58, 49))
(37, 4, Decimal('2000.00'), datetime.datetime(2022, 12, 3, 15, 58, 49))
(38, 5, Decimal('-3000.00'), datetime.datetime(2022, 12, 3, 15, 58, 49))
(39, 1, Decimal('3000.00'), datetime.datetime(2022, 12, 3, 15, 58, 49))
(64, 3, Decimal('-1000.00'), datetime.datetime(2022, 12, 3, 16, 1, 47))
(65, 1, Decimal('1000.00'), datetime.datetime(2022, 12, 3, 16, 1, 47))
(66, 3, Decimal('-2000.00'), datetime.datetime(2022, 12, 3, 16, 1, 47))
(67, 4, Decimal('2000.00'), datetime.datetime(2022, 12, 3, 16, 1, 47))
(68, 5, Decimal('-3000.00'), datetime.datetime

Parece ter funcionado... mas antes de executar o `COMMIT`, vamos testar o que acontece quando o saldo ficaria negativo:

In [18]:
movimentacao(1, -20000)

Executando query:


DatabaseError: 3819 (HY000): Check constraint 'c_saldo' is violated.

Opa, o constraint impediu o saque inválido! Excelente! Vamos reverter as ultimas mudanças antes de prosseguir:

In [19]:
db("ROLLBACK")

Executando query:


### Atividade

Realize as seguintes movimentações (com COMMIT se tudo funcionou dessa vez, e ROLLBACK se falhar):

| id_usuario | valor |
|--|--|
| 1 | +10000 |
| 2 | +4000 |
| 3 | +15000 |

In [20]:
db("START TRANSACTION;")
try:
    movimentacao(1, 1000)
    movimentacao(2, 4000)
    movimentacao(3, 15000)
    db("COMMIT;")
except Exception as e:
    print(e)
    db("ROLLBACK");

Executando query:
Executando query:
Executando query:
(1, 'Juca', 'Silva', Decimal('13000.00'))
Executando query:
Executando query:
(2, 'Mario', 'Ferreira', Decimal('4000.00'))
Executando query:
Executando query:
(3, 'Ana', 'Soares', Decimal('15800.00'))
Executando query:


Crie um trigger para realizar operações de transferência no contexto de um empréstimo.

**Resposta**: nesse contexto a operação é basicamente: já existe um empréstimo, vou aumentar mais.
```SQL
USE emprestimos;

DROP TRIGGER IF EXISTS transferencia;

DELIMITER //
CREATE TRIGGER transferencia 
BEFORE INSERT ON operacao
FOR EACH ROW
BEGIN
    UPDATE emprestimo
        SET valor_atual = valor_atual + NEW.valor  #valor da operacao - aumentando a dívida. Precisamos inserir mov.
            WHERE id_emprestimo = NEW.id_emprestimo;
    SELECT id_credor INTO @id_credor_var         
        FROM emprestimo
            WHERE id_emprestimo = NEW.id_emprestimo;
    SELECT id_devedor INTO @id_devedor_var
        FROM emprestimo
            WHERE id_emprestimo = NEW.id_emprestimo;
    INSERT INTO movimentacao(id_usuario, valor) VALUES (@id_credor_var, -NEW.valor);
    INSERT INTO movimentacao(id_usuario, valor) VALUES (@id_devedor_var, +NEW.valor);
    

END//

DELIMITER ;

```

## Views

*Views* são como tabelas *'lógicas'*, que são criadas através da aplicação de um comando `SELECT`. É como um `SELECT` pré-armazenado.

Vamos criar uma *view* para listar os nomes e sobrenomes de usuários, sem revelar seus saldos de conta:

```SQL
USE emprestimos;

CREATE VIEW nomes AS 
	SELECT DISTINCT nome, sobrenome FROM usuario;
```

Agora podemos usar esta view em consultas:

In [22]:
db("SELECT * from nomes;")

Executando query:
('Juca', 'Silva')
('Mario', 'Ferreira')
('Ana', 'Soares')
('Antonio', 'Reis')
('Paulo', 'Oliveira')
('Carla', 'Nakamura')
('Mario', 'Fontana')
('Max', 'Verstappen')
('Lando', 'Norris')
('Charles', 'Leclerc')
('Walter', 'Branco')


Porque usar views? Assim como no caso de stored procedures, podemos configurar permissões de acesso diferentes para esta view. Suponha que um vendedor deva ter acesso aos nomes dos clientes, mas não aos seus saldos (por razões de confidencialidade). Podemos conceder ao vendedor acesso apenas à essa view. Poderíamos ter resolvido o problema também com uma stored procedure: em SQL as coisas costumam ter várias soluções possíveis...

### Atividade

Crie uma view para mostrar, para cada usuário, a soma dos valores de emprestimo dos quais este cliente é credor. Se o cliente não tiver empréstimo associado como credor, o valor deve ser zero.

**Resposta:**
```mySQL

USE emprestimos

CREATE VIEW credor AS
    SELECT 
        id_usuario, SUM(IFNULL(valor_atual, 0.0)) as total
    FROM 
        usuario
        LEFT OUTER JOIN emprestimo ON usuario.id_usuario = emprestimo.id_credor
    GROUP BY
        id_usuario
```

Repita o processo para a posição de devedor.

**Resposta:**
```mySQL

USE emprestimos

CREATE VIEW devedor AS
    SELECT 
        id_usuario, SUM(IFNULL(valor_atual, 0.0)) as total
    FROM 
        usuario
        LEFT OUTER JOIN emprestimo ON usuario.id_usuario = emprestimo.id_devedor
    GROUP BY
        id_usuario
```

Crie uma view que lista o valor líquido de cada usuário, que é a soma dos valores como credor mais o saldo, subtraido da soma dos valores como devedor.

**Resposta:**
```mySQL

USE emprestimos

CREATE VIEW valor_liquido AS
    SELECT 
        id_usuario, nome, sobrenome, saldo + credor.total - devedor.total as valor
    FROM 
        usuario
        INNER JOIN credor USING (id_usuario)
        INNER JOIN devedor USING (id_usuario)
```

## Conclusão

Por hoje é só. Com a aula de hoje encerramos nossa discussão de SQL. Claro que o assunto ainda não se esgotou, mas agora vocês estão aptos a procurar detalhes de solução por conta própria em seus projetos - os principais conceitos foram cobertos. Deixamos de lado a investigação sobre *'cursores'* e laços, consulte o capítulo 15 do livro texto para ter uma noção de como estes artefatos são usados. (Não estamos cobrindo esse material por se tratar de conteúdo muito específico, e um tanto controverso!)

Nas próximas aulas vamos discutir alguns tópicos de teoria e implementação de sistemas de gerenciamento de banco de dados, para conhecer melhor como as coisas funcionam "por trás dos panos". Assim concluiremos o módulo 1 - bancos de dados relacionais e iniciamos o projeto 1.

Terminem todos os *handouts* e tragam suas dúvidas para o atendimento! Até a próxima!

In [23]:
connection.close()